<a href="https://colab.research.google.com/github/ariefanwar07/ariefanwar07.github.oi/blob/main/project1_indonesia_trade_indicators(1990%E2%80%932020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install requests pandas


In [9]:
import requests
import pandas as pd

# Function to fetch data from World Bank API
def fetch_world_bank_data(country, indicator, date_range):
    url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"
    params = {
        "date": date_range,
        "format": "json"
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            print(f"Data for {indicator} fetched successfully!")
            return pd.DataFrame(data[1])  # Data is in the second element of the response
        else:
            print(f"No data available for {indicator}.")
            return pd.DataFrame()
    else:
        print(f"Failed to fetch data for {indicator}: {response.status_code}")
        return pd.DataFrame()

# Fetch data for exports (US$), imports (US$), and trade balance (US$)
indicators = {
    "Exports (US$)": "NE.EXP.GNFS.CD",  # Exports of goods and services (current US$)
    "Imports (US$)": "NE.IMP.GNFS.CD",  # Imports of goods and services (current US$)
    "Trade Balance (US$)": "NE.RSB.GNFS.CD"  # Trade balance (US$)
}

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Loop through indicators and fetch data
for key, indicator in indicators.items():
    df = fetch_world_bank_data("ID", indicator, "1990:2020")
    if not df.empty:
        df = df[["date", "value"]].rename(columns={"date": "Year", "value": key})
        if all_data.empty:
            all_data = df
        else:
            all_data = pd.merge(all_data, df, on="Year", how="outer")

# Display the final DataFrame
print(all_data)

# Save to CSV
all_data.to_csv("indonesia_trade_data.csv", index=False)
print("Data saved to indonesia_trade_data.csv")


Data for NE.EXP.GNFS.CD fetched successfully!
Data for NE.IMP.GNFS.CD fetched successfully!
Data for NE.RSB.GNFS.CD fetched successfully!
    Year  Exports (US$)  Imports (US$)  Trade Balance (US$)
0   1990   2.898253e+10   2.715728e+10         1.825256e+09
1   1991   3.306381e+10   3.089119e+10         2.172618e+09
2   1992   3.880173e+10   3.472088e+10         4.080850e+09
3   1993   4.227440e+10   3.755594e+10         4.718461e+09
4   1994   4.689663e+10   4.486988e+10         2.026750e+09
5   1995   5.318531e+10   5.588228e+10        -2.696967e+09
6   1996   5.871720e+10   6.011698e+10        -1.399778e+09
7   1997   6.010604e+10   6.070015e+10        -5.941129e+08
8   1998   5.055573e+10   4.124971e+10         9.306014e+09
9   1999   4.972026e+10   3.840207e+10         1.131819e+10
10  2000   6.762117e+10   5.026469e+10         1.735648e+10
11  2001   6.262588e+10   4.935520e+10         1.327068e+10
12  2002   6.395680e+10   5.163844e+10         1.231836e+10
13  2003   7.155314e+1

In [20]:
import pandas as pd
import altair as alt

# Load the data
data = pd.read_csv("indonesia_trade_data.csv")  # Replace with your actual data file path

# Convert Year to datetime format before melting
data['Year'] = pd.to_datetime(data['Year'], format='%Y')

# Melt the DataFrame into a long format for Altair compatibility
data_long = data.melt(id_vars=["Year"], value_vars=["Exports (US$)", "Imports (US$)", "Trade Balance (US$)"],
                      var_name="Indicator", value_name="Value")

# Create the static interactive line chart
chart = alt.Chart(data_long).mark_line(point=True).encode(
    x=alt.X("Year:T", title="Year"),  # Use temporal type for the x-axis
    y=alt.Y("Value:Q", title="Value (US$)", scale=alt.Scale(zero=False)),
    color="Indicator:N",
    tooltip=["Year:T", "Indicator:N", "Value:Q"]
).properties(
    title="Indonesia Trade Indicators Over Time (1990–2020)",
    width=500,  # Reduced width
    height=300  # Reduced height
)

# Save the chart as JSON
chart.save("project1_indonesia_trade_data.json")

print("project1_indonesia_trade_data saved as 'project1_indonesia_trade_data.json'.")


project1_indonesia_trade_data saved as 'project1_indonesia_trade_data.json'.
